In [6]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Investigation of the _Synchronized Brainwave Dataset_

Data set URL: [https://www.kaggle.com/berkeley-biosense/synchronized-brainwave-dataset](https://www.kaggle.com/berkeley-biosense/synchronized-brainwave-dataset)

The data set consists of EEG readings, using a one-channel consumer EEG device called [NeuroSky](http://neurosky.com/), from 30 participants. Two slightly different stimuli were presented to two (mutual exclusive) groups of 15 participants each. The two stiumuli were videos accompanied with voice instructions presenting different tasks, interleaved with phases of relaxation (with eyes shut) and hearing music.
The data from the EEG head bands was collected on a central server that keept track of the latency so that later the **synchronized timestamps** could be computed. Each row in the `eeg-data.csv` file represents one data packet recieved by the server.